In [75]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.callbacks import TensorBoard
import pandas as pd
import numpy as np
from multiprocessing import  Pool
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from time import time

In [85]:
print("open dataset")
train = pd.read_csv("../dataset/goodreads_train.csv")
# test = pd.read_csv("dataset/goodreads_test.csv")

open dataset


In [65]:
nb_note =  train["n_votes"]

In [66]:
nb_note

0         28
1          1
2         22
3          5
4          9
          ..
899995     0
899996     1
899997     1
899998     0
899999     0
Name: n_votes, Length: 900000, dtype: int64

In [51]:
y_train = train['rating']

In [52]:
train_prepro = pd.DataFrame(data=np.load(file="../prepro_train_archive.npy", allow_pickle=True), columns=['review_text'])['review_text']

In [53]:
"""dic = {}
all_stop_word = stopwords.words("english")
for stop_word in all_stop_word:
    dic[stop_word] = stop_word
def parallelize_dataframe(df, func, n_cores=4):
    print("split")

    df_split = np.array_split(df, n_cores)
    del df
    print(df_split)

    with Pool(n_cores) as pool:
        df = pd.concat(pool.map(func, df_split))

    return df
def prepro(text):
    words = text.lower()
    tokens = nltk.word_tokenize(words)
    words_stop_less = [w for w in tokens if dic.get(w) == None]
    stemmed = [PorterStemmer().stem(w) for w in words_stop_less]
    return " ".join(stemmed)

def prepro_map(data_frame):
    tqdm.pandas()
    return data_frame.progress_apply(lambda x: prepro(x))"""

'dic = {}\nall_stop_word = stopwords.words("english")\nfor stop_word in all_stop_word:\n    dic[stop_word] = stop_word\ndef parallelize_dataframe(df, func, n_cores=4):\n    print("split")\n\n    df_split = np.array_split(df, n_cores)\n    del df\n    print(df_split)\n\n    with Pool(n_cores) as pool:\n        df = pd.concat(pool.map(func, df_split))\n\n    return df\ndef prepro(text):\n    words = text.lower()\n    tokens = nltk.word_tokenize(words)\n    words_stop_less = [w for w in tokens if dic.get(w) == None]\n    stemmed = [PorterStemmer().stem(w) for w in words_stop_less]\n    return " ".join(stemmed)\n\ndef prepro_map(data_frame):\n    tqdm.pandas()\n    return data_frame.progress_apply(lambda x: prepro(x))'

In [67]:
inputs = keras.Input(shape=(1,), dtype=tf.string)
#inputs2 = keras.Input(shape=(1), dtype=tf.int64)

vectorize_layer = keras.layers.TextVectorization(
    max_tokens= 494871,
    standardize='lower_and_strip_punctuation',
    split='whitespace',
    output_mode='int',
    output_sequence_length=1400,
    vocabulary=np.load('../voc2.npy')
)(inputs)

#conc = keras.layers.concatenate([vectorize_layer, inputs2])

layer1 = keras.layers.Dense(64, activation=tf.keras.activations.relu)(vectorize_layer)
layer2 = keras.layers.Dense(128, activation=tf.keras.activations.relu)(layer1)
layer3 = keras.layers.Dense(64, activation=tf.keras.activations.relu)(layer2)
outputs = keras.layers.Dense(1, activation=tf.keras.activations.relu)(layer3)

In [68]:
model = keras.Model(inputs=inputs, outputs=outputs, name="mnist_model")
tensorboard = TensorBoard(log_dir="../logs/relu".format(time()))

In [69]:
keras.utils.plot_model(model, "my_first_model.png")

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [70]:
keras.utils.plot_model(model, "my_first_model.png")

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [71]:
model.summary()

Model: "mnist_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_7 (TextV  (None, 1400)             0         
 ectorization)                                                   
                                                                 
 dense_28 (Dense)            (None, 64)                89664     
                                                                 
 dense_29 (Dense)            (None, 128)               8320      
                                                                 
 dense_30 (Dense)            (None, 64)                8256      
                                                                 
 dense_31 (Dense)            (None, 1)                 65        
                                                       

In [73]:
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
              loss=tf.keras.losses.mean_absolute_percentage_error,
              metrics=[tf.keras.metrics.Accuracy()]
              )

In [74]:
model.fit(train_prepro, y_train, epochs=20,
                  callbacks=[
                      tf.keras.callbacks.TensorBoard(log_dir="../logs/relu"),
                  ],
                  batch_size=50000
                  )

Epoch 1/20
18/18 [==============================] - 24s 1s/step - loss: 2260461312.0000 - accuracy: 0.0326
Epoch 2/20
 1/18 [>.............................] - ETA: 20s - loss: 96.6040 - accuracy: 0.0340

KeyboardInterrupt: 

In [15]:
#model.save("../models_trained/linear_model_2")

In [86]:
output = tf.keras.utils.to_categorical(train["rating"], num_classes=6, dtype ="int32")

In [87]:
output

array([[0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0],
       ...,
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0]])